In [1]:
import gensim
import os

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
from nltk.corpus import PlaintextCorpusReader as PCR

In [4]:
os.chdir(os.path.expanduser(os.path.join('~', 'sch_corpus')))

In [5]:
hmong = PCR('.', '.*').sents()

In [6]:
import nltk

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nathan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
sentences = list(hmong)

In [8]:
os.chdir(os.path.expanduser(os.path.join('~', 
                                         'python_workspace', 
                                         'ner_sent_embeddings', 
                                         'ner_tagged_April_2020_version', 
                                         'anatomy_tagged')))

In [9]:
filenames = os.listdir()

In [10]:
data_raw = []

In [11]:
for file in filenames:
    f = open(file, 'r')
    content = f.readlines()
    data_raw.append(content[1:])

In [12]:
len(data_raw[3])

2

In [12]:
import re

In [13]:
processed_data = []
for data_portion in data_raw:
    for line in data_portion:
        split_line = line.split(' ')
        processed_line = []
        for word in split_line:
            processed_line.append(re.sub('▹.*', '', word))
        processed_data.append(processed_line)        

In [14]:
sentence_data = []
for portion in processed_data:
    current_sentence = []
    for word in portion:
        if word not in ['.', '?', '!']:
            current_sentence.append(word)
        else:
            current_sentence.append(word)
            sentence_data.append(current_sentence)
            current_sentence = []

In [15]:
questions = []
for item in sentences:
    if item[-1] == '?':
        questions.append(item)

In [17]:
# each of these can be labeled 'other'
len(questions)

55375

In [23]:
for item in questions:
    if 'tsos' in item:
        print(item)

['Yus', 'hnab', '-', 'tsos', 'puas', 'puv', 'tsaus', 'yuav', 'mus', 'nkag', 'tau', 'nrog', 'lub', 'saangkhoun', '-', 'society', '-', 'tshiab', 'tim', 'ub', '?']
['ntshai', 'ho', 'tshuav', 'sab', 'hauv', 'yus', 'tsis', 'pom', 'xwb', 'lauv', '....', 'tej', 'tus', 'mas', 'xyov', 'xyov', 'laud', '...', 'xyov', 'muab', 'tej', 'qhov', 'maug', 'tsos', 'qhov', 'twg', 'lawm', 'laud', '?']
['Peb', 'tuwb', 'paub', 'YTK', 'zoo', 'laid', 'thiab', 'kuv', 'cov', 'neej', 'tsa', 'Hmoob', 'Yaj', 'tias', 'YTK', 'muab', 'tsos', 'dub', 'laid', '?']
['Nej', 'cov', 'mus', 'noj', '30', 'tag', 'los', 'nej', 'puas', 'muaj', 'tsos', 'nco', 'nco', 'li', 'ntawd', 'thiab', 'na', '?']
['Vaaj', 'aws', '...', 'NkaujNyiag', 'twb', 'hastas', 'nwg', 'nim', 'nrug', 'cov', 'koomhum', 'lujx', 'xwsle', 'Amnesty', 'International', ',', 'DOS', ',', 'US', 'Ambassador', ',', 'Lao', 'PDR', 'hab', 'cov', 'qib', 'sab', 'tham', 'lawm', 'xwb', 'mas', 'nwg', 'zoosab', 'heev', 'vim', 'Nplog', 'yuav', 'tsos', 'puab', 'tsis', 'muab', 'pu

In [16]:
sch_sentences = [gensim.models.doc2vec.TaggedDocument(line, [i]) for i, line in enumerate(sentences)]

In [17]:
hmc_sentences = [gensim.models.doc2vec.TaggedDocument(line, [i]) for i, line in enumerate(sentence_data)]

In [18]:
os.chdir(os.path.join('..', '..'))

In [19]:
question_examples = []
question_labels = []
f = open('question_type_training_set_2.txt', 'r')
other_questions_raw = f.readlines()
for item in other_questions_raw:
    item_ = re.sub('\?', ' ?', item)
    question_examples.append(item_.strip().split(' ')[:-2])
    question_labels.append('Other')
f.close()

In [20]:
f = open('symptom_questions.txt', 'r')
symptom_questions_raw = f.readlines()
for item in symptom_questions_raw:
    question_examples.append(item.strip().split(' ')[:-2])
    question_labels.append('Symptom')
f.close()

In [21]:
os.chdir(os.path.join('hmong_medical_data_sources'))

In [22]:
additional_examples = []
additional_labels = []
# these are additional examples taken from other government medical documents
f = open('additional_training_data.txt', 'r')
additional_questions_raw = f.readlines()
for item in additional_questions_raw:
    item_ = item.strip().split(' ')
    additional_examples.append(item_[:-2])
    additional_labels.append(item_[-1])
f.close()
f = open('additional_data_third.txt', 'r')
additional_questions_raw = f.readlines()
for item in additional_questions_raw:
    item_ = item.strip().split(' ')
    additional_examples.append(item_[:-2])
    additional_labels.append(item_[-1])
f.close()

In [23]:
total_examples = question_examples + additional_examples

In [24]:
f = open('question_examples.txt', 'w')
for i, item in enumerate(question_examples):
    f.write(' '.join(item))
    f.write(' | ')
    f.write(question_labels[i])
    f.write('\n')
for i, item in enumerate(additional_examples):
    f.write(' '.join(item))
    f.write(' | ')
    f.write(additional_labels[i])
    f.write('\n')
f.close()

In [25]:
import mysql.connector

In [26]:
cnx = mysql.connector.connect(user='hadoop', password='lK3pA!2j', host="127.0.0.1", database="ner")

In [27]:
cursor = cnx.cursor()

In [28]:
# this part is where I load the templates and populate them with disease names
sql_cmd = """SELECT name FROM dsynNames;"""
cursor.execute(sql_cmd)

dsynNames = []
for item in cursor:
    dsynNames.append(item[0])

In [29]:
os.chdir('..')
f = open('symptom_templates.txt', 'r')
templates = f.readlines()

generated_questions = []
for item in templates:
    for name in dsynNames:
        # use re.sub to replace [name]
        new_q = re.sub('\[name\]', name, item)
        # remember to get rid of 'mob mob' instances after this
        new_q = re.sub('mob mob', 'mob', new_q)
        generated_questions.append(new_q)
f.close()

In [30]:
generated_questions = list(set(generated_questions))

In [31]:
f = open('generated_questions.txt', 'w')
for item in generated_questions:
    f.write(item)
f.close()

In [32]:
generated_questions_final = []
generated_labels_final = []
for item in generated_questions:
    item_ = item.strip().split(' ')
    generated_questions_final.append(item_[:-2])
    generated_labels_final.append(item_[-1])

In [33]:
total_examples += generated_questions_final

In [34]:
question_sentences = [gensim.models.doc2vec.TaggedDocument(line, [i]) for i, line in enumerate(total_examples)]

In [35]:
sentence_base = sch_sentences + hmc_sentences + question_sentences

In [ ]:
doc_model_full = gensim.models.Doc2Vec(min_count=1, iter=20, vector_size=32)

In [44]:
doc_model_full.build_vocab(sentence_base)

INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #10000, processed 277887 words (2123543/s), 20013 word types, 10000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #20000, processed 556633 words (2866354/s), 27176 word types, 20000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #30000, processed 819487 words (3043737/s), 31777 word types, 30000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #40000, processed 1080151 words (3047149/s), 36907 word types, 40000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #50000, processed 1354183 words (3110326/s), 40388 word types, 50000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #60000, processed 1620018 words (2666860/s), 43280 word types, 60000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #70000, processed 1902993 words (2912023/s), 45973 word type

INFO:gensim.models.doc2vec:PROGRESS: at example #660000, processed 16544801 words (2976380/s), 139987 word types, 660000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #670000, processed 16817779 words (2886088/s), 146346 word types, 669319 tags
INFO:gensim.models.doc2vec:collected 146916 word types and 669319 unique tags from a corpus of 678467 examples and 16961998 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=1 retains 146916 unique words (100% of original 146916, drops 0)
INFO:gensim.models.word2vec:effective_min_count=1 leaves 16961998 word corpus (100% of original 16961998, drops 0)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 146916 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 65 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 12786947 word corpus (75.4% of prior 16961998)
INFO:gensim.models.base_any2vec:estimated required memory for 146916 words an

In [45]:
doc_model_full.train(sch_sentences, total_examples=len(sch_sentences), epochs=20)

INFO:gensim.models.base_any2vec:training model with 3 workers on 146916 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.94% examples, 139189 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.51% examples, 187745 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 3.92% examples, 190372 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 5.50% examples, 198709 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 7.21% examples, 207426 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 8.89% examples, 211439 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 10.41% examples, 213498 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 12.06% examples, 2

INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 18.24% examples, 231817 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 19.84% examples, 230309 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 21.45% examples, 230262 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 23.18% examples, 230315 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 24.90% examples, 229110 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 26.53% examples, 228098 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 28.25% examples, 227857 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 29.86% examples, 226659 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 31.58% examples, 226554 words/s, in_qsize 5, out_

INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 38.17% examples, 224375 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 39.94% examples, 223581 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 41.62% examples, 223249 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 43.38% examples, 222619 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 45.10% examples, 222515 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 46.85% examples, 221969 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 48.61% examples, 221375 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 50.33% examples, 220797 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 3 - PROGRESS: at 52.19% examples, 220255 words/s, in_qsize 5, out_

INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 58.40% examples, 220207 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 60.15% examples, 219892 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 61.83% examples, 219476 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 63.54% examples, 219509 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 65.27% examples, 219283 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 66.97% examples, 219060 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 68.73% examples, 218993 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 70.43% examples, 219056 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 4 - PROGRESS: at 72.19% examples, 218998 words/s, in_qsize 6, out_

INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 78.87% examples, 218708 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 80.53% examples, 219028 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 82.20% examples, 219283 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 83.89% examples, 219528 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 85.58% examples, 219769 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 87.26% examples, 219547 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 88.93% examples, 219799 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 90.58% examples, 219834 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 92.25% examples, 219977 words/s, in_qsize 5, out_

...

INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 29.80% examples, 226618 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 31.44% examples, 226408 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 33.19% examples, 225342 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 34.96% examples, 225082 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 36.63% examples, 225118 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 38.37% examples, 224739 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 40.09% examples, 224147 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 41.83% examples, 223677 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 16 - PROGRESS: at 43.57% examples, 223207 words/s, in_qsiz

INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 47.34% examples, 222215 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 49.10% examples, 221697 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 50.80% examples, 221034 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 52.61% examples, 220477 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 54.46% examples, 219592 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 56.15% examples, 219207 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 58.01% examples, 218585 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 59.79% examples, 218321 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 17 - PROGRESS: at 61.50% examples, 218182 words/s, in_qsiz

INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 64.49% examples, 217129 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 66.21% examples, 216968 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 67.94% examples, 216916 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 69.59% examples, 216893 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 71.32% examples, 216745 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 73.02% examples, 216698 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 74.69% examples, 216483 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 76.44% examples, 216671 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 18 - PROGRESS: at 78.16% examples, 216669 words/s, in_qsiz

INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 81.62% examples, 217926 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 83.29% examples, 218112 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 84.97% examples, 218355 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 86.66% examples, 218381 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 88.33% examples, 218491 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 90.04% examples, 218752 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 91.76% examples, 218865 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 93.47% examples, 218921 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 19 - PROGRESS: at 95.22% examples, 219183 words/s, in_qsiz

INFO:gensim.models.base_any2vec:EPOCH 20 - PROGRESS: at 99.59% examples, 219415 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 20 : training on 16800887 raw words (13333105 effective words) took 60.7s, 219551 effective words/s
INFO:gensim.models.base_any2vec:training on a 336017740 raw words (266660339 effective words) took 1211.4s, 220122 effective words/s


In [46]:
doc_model_full.train(hmc_sentences, total_examples=len(hmc_sentences), epochs=20)

INFO:gensim.models.base_any2vec:training model with 3 workers on 146916 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 89279 raw words (70894 effective words) took 0.3s, 216315 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 89279 raw words (71155 effective words) took 0.3s, 223359 effective words/s
INFO:gensim.models.base_any2vec:worker thread f

INFO:gensim.models.base_any2vec:training on a 1785580 raw words (1420949 effective words) took 7.5s, 190499 effective words/s


In [47]:
doc_model_full.train(question_sentences, total_examples=len(question_sentences), epochs=20)

INFO:gensim.models.base_any2vec:training model with 3 workers on 146916 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 71832 raw words (61825 effective words) took 0.4s, 147329 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 71832 raw words (61808 effective words) took 0.4s, 150671 effective words/s
INFO:gensim.models.base_any2vec:worker thread f

INFO:gensim.models.base_any2vec:training on a 1436640 raw words (1236910 effective words) took 8.7s, 142569 effective words/s


In [48]:
doc_model_full.save('doc_model_full.h5')

INFO:gensim.utils:saving Doc2Vec object under doc_model_full.h5, separately None
INFO:gensim.utils:storing np array 'vectors_docs' to doc_model_full.h5.docvecs.vectors_docs.npy
INFO:gensim.utils:saved doc_model_full.h5


In [50]:
doc_model_hmc = gensim.models.Doc2Vec(min_count=1, iter=20, vector_size=32)
doc_model_hmc.build_vocab(hmc_sentences + question_sentences)

INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 2850 word types and 5515 unique tags from a corpus of 9148 examples and 161111 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=1 retains 2850 unique words (100% of original 2850, drops 0)
INFO:gensim.models.word2vec:effective_min_count=1 leaves 161111 word corpus (100% of original 161111, drops 0)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 2850 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 56 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 90692 word corpus (56.3% of prior 161111)
INFO:gensim.models.base_any2vec:estimated required memory for 2850 words and 32 dimensions: 2860520 bytes
INFO:gensim.models.word2vec:resetting layer weights


In [51]:
doc_model_hmc.train(hmc_sentences, total_examples=len(hmc_sentences), epochs=20)
doc_model_hmc.train(question_sentences, total_examples=len(question_sentences), epochs=20)

INFO:gensim.models.base_any2vec:training model with 3 workers on 2850 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 89279 raw words (62861 effective words) took 0.5s, 133800 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 89279 raw words (63114 effective words) took 0.3s, 219343 effective words/s
INFO:gensim.models.base_any2vec:worker thread fin

INFO:gensim.models.base_any2vec:training on a 1785580 raw words (1259904 effective words) took 6.4s, 198271 effective words/s
INFO:gensim.models.base_any2vec:training model with 3 workers on 2850 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 71832 raw words (36849 effective words) took 0.4s, 99682 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 

INFO:gensim.models.base_any2vec:EPOCH - 20 : training on 71832 raw words (36590 effective words) took 0.4s, 93858 effective words/s
INFO:gensim.models.base_any2vec:training on a 1436640 raw words (736293 effective words) took 8.0s, 92451 effective words/s


In [52]:
doc_model_hmc.save('doc_model_hmc.h5')

INFO:gensim.utils:saving Doc2Vec object under doc_model_hmc.h5, separately None
INFO:gensim.utils:saved doc_model_hmc.h5


In [53]:
doc_model_qs = gensim.models.Doc2Vec(min_count=1, iter=20, vector_size=32)
doc_model_qs.build_vocab(question_sentences)

/home/nathan/miniconda2/lib/python3.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 948 word types and 5515 unique tags from a corpus of 5515 examples and 71832 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=1 retains 948 unique words (100% of original 948, drops 0)
INFO:gensim.models.word2vec:effective_min_count=1 leaves 71832 word corpus (100% of original 71832, drops 0)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 948 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 46 most-common words
INFO:gensim.mo

In [55]:
doc_model_qs.train(question_sentences, total_examples=len(question_sentences), epochs=20)

INFO:gensim.models.base_any2vec:training model with 3 workers on 948 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 71832 raw words (33879 effective words) took 0.3s, 113928 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 71832 raw words (33734 effective words) took 0.4s, 84448 effective words/s
INFO:gensim.models.base_any2vec:worker thread finis

In [56]:
doc_model_qs.save('doc_model_qs.h5')

INFO:gensim.utils:saving Doc2Vec object under doc_model_qs.h5, separately None
INFO:gensim.utils:saved doc_model_qs.h5


In [36]:
doc_model_full = gensim.models.Doc2Vec.load('doc_model_full.h5')
doc_model_hmc = gensim.models.Doc2Vec.load('doc_model_hmc.h5')
doc_model_qs = gensim.models.Doc2Vec.load('doc_model_qs.h5')

INFO:gensim.utils:loading Doc2Vec object from doc_model_full.h5
INFO:gensim.utils:loading vocabulary recursively from doc_model_full.h5.vocabulary.* with mmap=None
INFO:gensim.utils:loading trainables recursively from doc_model_full.h5.trainables.* with mmap=None
INFO:gensim.utils:loading wv recursively from doc_model_full.h5.wv.* with mmap=None
INFO:gensim.utils:loading docvecs recursively from doc_model_full.h5.docvecs.* with mmap=None
INFO:gensim.utils:loading vectors_docs from doc_model_full.h5.docvecs.vectors_docs.npy with mmap=None
INFO:gensim.utils:loaded doc_model_full.h5
INFO:gensim.utils:loading Doc2Vec object from doc_model_hmc.h5
INFO:gensim.utils:loading vocabulary recursively from doc_model_hmc.h5.vocabulary.* with mmap=None
INFO:gensim.utils:loading trainables recursively from doc_model_hmc.h5.trainables.* with mmap=None
INFO:gensim.utils:loading wv recursively from doc_model_hmc.h5.wv.* with mmap=None
INFO:gensim.utils:loading docvecs recursively from doc_model_hmc.h5.d

In [37]:
# I have no idea what any of this is below here
# I think it prepps the dataset for what comes next
dataset = []
for item in other_questions_raw:
    item_ = re.sub('\?', ' ?', item).strip().split(' ')
    dataset.append((item_[:-2], item_[-1]))
for item in symptom_questions_raw:
    item_ = item.strip().split(' ')
    dataset.append((item_[:-2], item_[-1]))

In [38]:
for item in generated_questions:
    item_ = item.strip().split(' ')
    dataset.append((item_[:-2], item_[-1]))

In [39]:
for item in questions:
    dataset.append((item, 'Other'))

In [40]:
dataset[-5:]

[(['nrab',
   '6lawv',
   'ualawvjkuj',
   'ua',
   'yog',
   'hnubno',
   'koj',
   'mus',
   'ncig',
   'topm',
   'park',
   'koj',
   'pom',
   'ib',
   'daim',
   '100',
   'dollar',
   'bill',
   'koj',
   'puas',
   'nqa',
   'mus',
   'tshaj',
   'tawm',
   'tom',
   'radio',
   'los',
   'koj',
   'yuav',
   'muab',
   'cioa',
   'tau',
   'SIV',
   'no',
   'ma',
   '?'],
  'Other'),
 (['yug',
   'tus',
   'puav',
   'ntxhaiskom',
   'pob',
   'twDAV2',
   'ces',
   'COMMIE',
   'yuav',
   'nce',
   'rooblawvkawg',
   'tuaj',
   'ntawm',
   '3',
   'tus',
   'navxwj',
   'zos',
   'tuaj',
   'ces',
   'MAG',
   'NTXIAB',
   'ntawm',
   'tus',
   'ntxhais',
   'POB',
   'TW',
   'DAV2',
   'COMMIE',
   'KHAV2kwv',
   'rab',
   'AK',
   'kom',
   'DAJ',
   'thiab',
   'LIAB',
   'TSHAJ',
   'PLAWS',
   'nrog',
   'B51',
   'rablub',
   'pob',
   'tsawb',
   'LOJ2',
   'tuajkhav',
   'theeb',
   '6',
   'peb',
   'tus',
   'ntxhais',
   'pobtw',
   'DAV2',
   'tias',
   'tus',
 

In [41]:
import numpy as np

In [42]:
X_in, y = zip(*dataset)

In [44]:
vectors_full = []
vectors_hmc = []
vectors_qs = []
for item in X_in:
    vectors_full.append(doc_model_full.infer_vector(item))
    vectors_hmc.append(doc_model_hmc.infer_vector(item))
    vectors_qs.append(doc_model_qs.infer_vector(item))
X_full = np.array(vectors_full)
X_hmc = np.array(vectors_hmc)
X_qs = np.array(vectors_qs)

In [47]:
from tensorflow import keras

In [49]:
from tensorflow.keras.utils import to_categorical

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
y_convert = {k: i for (i, k) in enumerate(set(y))}

In [52]:
y_convert['Anatomy'] = 2

In [53]:
y_convert

{'Symptom': 0, 'Other': 1, 'Anatomy': 2}

In [54]:
y_cat = to_categorical([y_convert[word] for word in y], num_classes=3)

In [55]:
y_cat[0, 1]

1.0

In [56]:
def label_smoother(y_data, factor):
    margin = factor / len(y_data[0])
    for i, item in enumerate(y_data):
        argmax = np.argmax(y_data[i])
        margin = factor / len(y_data[i])
        for j, value in enumerate(item):
            if j == argmax:
                y_data[i, j] = y_data[i, j] - factor + margin
            else:
                y_data[i, j] = margin

In [57]:
label_smoother(y_cat, 0.1)

In [58]:
y_cat[0]

array([0.03333334, 0.93333334, 0.03333334], dtype=float32)

In [60]:
X_full_train, X_full_test, X_hmc_train, X_hmc_test, X_qs_train, X_qs_test, y_train, y_test = train_test_split(X_full, X_hmc, X_qs, y_cat, test_size=0.1)

In [62]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l1_l2, l2

In [64]:
input_ = Input(shape=(len(vectors_full[0]),))
dense1 = Dense(128, activation='relu')(input_)
dropout = Dropout(0.1)(dense1)
dense2 = Dense(128, activation='relu')(dropout)
dense3 = Dense(3, activation='softmax')(dense2)

model = Model(inputs=input_, outputs=dense3)

In [65]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 21,123
Trainable params: 21,123
Non-trainable params: 0
_________________________________________________________________


In [67]:
from tensorflow.keras.losses import CategoricalCrossentropy

In [68]:
cc = CategoricalCrossentropy(label_smoothing=0.03)

In [69]:
# how to get precision and recall?
model.compile(loss=cc, optimizer='adam', metrics=['accuracy'])

In [70]:
# incredibly high degree of variance
# could pull examples from sch_corpus for other, other medical docs for symptoms, body parts
# body parts are still totally missing
callbacks = model.fit(X_full_train, y_train, epochs=100, validation_split=0.15)

Epoch 1/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.4006 - accuracy: 0.9721 - val_loss: 0.3719 - val_accuracy: 0.9879
Epoch 2/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3673 - accuracy: 0.9897 - val_loss: 0.3655 - val_accuracy: 0.9896
Epoch 3/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3642 - accuracy: 0.9912 - val_loss: 0.3651 - val_accuracy: 0.9910
Epoch 4/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3624 - accuracy: 0.9921 - val_loss: 0.3657 - val_accuracy: 0.9904
Epoch 5/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3612 - accuracy: 0.9928 - val_loss: 0.3614 - val_accuracy: 0.9931
Epoch 6/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3600 - accuracy: 0.9939 - val_loss: 0.3606 - val_accuracy: 0.9934
Epoch 7/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3596 - accuracy: 0.9939 - val_loss: 0.3610 - val_ac

Epoch 57/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3517 - accuracy: 0.9986 - val_loss: 0.3573 - val_accuracy: 0.9951
Epoch 58/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3514 - accuracy: 0.9986 - val_loss: 0.3566 - val_accuracy: 0.9954
Epoch 59/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3516 - accuracy: 0.9985 - val_loss: 0.3580 - val_accuracy: 0.9951
Epoch 60/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3515 - accuracy: 0.9986 - val_loss: 0.3556 - val_accuracy: 0.9962
Epoch 61/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3518 - accuracy: 0.9985 - val_loss: 0.3576 - val_accuracy: 0.9949
Epoch 62/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3513 - accuracy: 0.9986 - val_loss: 0.3555 - val_accuracy: 0.9966
Epoch 63/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3510 - accuracy: 0.9988 - val_loss: 0.3563 -

In [71]:
model.evaluate(X_full_test, y_test)

191/191 [==============================] - 1s 4ms/step - loss: 0.3577 - accuracy: 0.9956


[0.35770344734191895, 0.995561420917511]

In [73]:
input_ = Input(shape=(len(vectors_hmc[0]),))
dense1 = Dense(128, activation='relu')(input_)
dropout = Dropout(0.1)(dense1)
dense2 = Dense(128, activation='relu')(dropout)
dense3 = Dense(3, activation='softmax')(dense2)

model_hmc = Model(inputs=input_, outputs=dense3)

In [74]:
model_hmc.compile(loss=cc, optimizer='adam', metrics=['accuracy'])

In [75]:
callbacks = model_hmc.fit(X_hmc_train, y_train, epochs=100, validation_split=0.15)

Epoch 1/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.4442 - accuracy: 0.9405 - val_loss: 0.4092 - val_accuracy: 0.9605
Epoch 2/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3965 - accuracy: 0.9718 - val_loss: 0.3846 - val_accuracy: 0.9804
Epoch 3/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3811 - accuracy: 0.9829 - val_loss: 0.3789 - val_accuracy: 0.9820
Epoch 4/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3727 - accuracy: 0.9873 - val_loss: 0.3742 - val_accuracy: 0.9853
Epoch 5/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3688 - accuracy: 0.9890 - val_loss: 0.3703 - val_accuracy: 0.9867
Epoch 6/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3667 - accuracy: 0.9906 - val_loss: 0.3688 - val_accuracy: 0.9876
Epoch 7/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3651 - accuracy: 0.9917 - val_loss: 0.3705 - val_ac

Epoch 57/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3513 - accuracy: 0.9991 - val_loss: 0.3639 - val_accuracy: 0.9920
Epoch 58/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3514 - accuracy: 0.9989 - val_loss: 0.3653 - val_accuracy: 0.9910
Epoch 59/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3516 - accuracy: 0.9989 - val_loss: 0.3642 - val_accuracy: 0.9912
Epoch 60/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3519 - accuracy: 0.9986 - val_loss: 0.3650 - val_accuracy: 0.9911
Epoch 61/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3516 - accuracy: 0.9989 - val_loss: 0.3640 - val_accuracy: 0.9920
Epoch 62/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3518 - accuracy: 0.9986 - val_loss: 0.3654 - val_accuracy: 0.9918
Epoch 63/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3515 - accuracy: 0.9990 - val_loss: 0.3650 -

In [77]:
model_hmc.evaluate(X_hmc_test, y_test)

191/191 [==============================] - 1s 3ms/step - loss: 0.3607 - accuracy: 0.9931


[0.36066102981567383, 0.9930955171585083]

In [79]:
input_ = Input(shape=(len(vectors_hmc[0]),))
dense1 = Dense(128, activation='relu')(input_)
dropout = Dropout(0.1)(dense1)
dense2 = Dense(128, activation='relu')(dropout)
dense3 = Dense(3, activation='softmax')(dense2)

model_qs = Model(inputs=input_, outputs=dense3)

In [80]:
model_qs.compile(loss=cc, optimizer='adam', metrics=['accuracy'])

In [81]:
callbacks = model_qs.fit(X_qs_train, y_train, epochs=100, validation_split=0.15)

Epoch 1/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.4233 - accuracy: 0.9580 - val_loss: 0.3798 - val_accuracy: 0.9826
Epoch 2/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3778 - accuracy: 0.9849 - val_loss: 0.3723 - val_accuracy: 0.9893
Epoch 3/100
1455/1455 [==============================] - 10s 7ms/step - loss: 0.3696 - accuracy: 0.9895 - val_loss: 0.3664 - val_accuracy: 0.9917
Epoch 4/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3667 - accuracy: 0.9912 - val_loss: 0.3643 - val_accuracy: 0.9923
Epoch 5/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3637 - accuracy: 0.9926 - val_loss: 0.3637 - val_accuracy: 0.9926
Epoch 6/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3620 - accuracy: 0.9934 - val_loss: 0.3615 - val_accuracy: 0.9943
Epoch 7/100
1455/1455 [==============================] - 10s 7ms/step - loss: 0.3613 - accuracy: 0.9940 - val_loss: 0.3606 - val_

Epoch 57/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3507 - accuracy: 0.9993 - val_loss: 0.3581 - val_accuracy: 0.9949
Epoch 58/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3509 - accuracy: 0.9993 - val_loss: 0.3571 - val_accuracy: 0.9951
Epoch 59/100
1455/1455 [==============================] - 8s 5ms/step - loss: 0.3507 - accuracy: 0.9993 - val_loss: 0.3576 - val_accuracy: 0.9953
Epoch 60/100
1455/1455 [==============================] - 7s 5ms/step - loss: 0.3510 - accuracy: 0.9991 - val_loss: 0.3573 - val_accuracy: 0.9955
Epoch 61/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3510 - accuracy: 0.9991 - val_loss: 0.3574 - val_accuracy: 0.9954
Epoch 62/100
1455/1455 [==============================] - 8s 6ms/step - loss: 0.3507 - accuracy: 0.9993 - val_loss: 0.3571 - val_accuracy: 0.9956
Epoch 63/100
1455/1455 [==============================] - 9s 6ms/step - loss: 0.3508 - accuracy: 0.9991 - val_loss: 0.3569 -

In [82]:
model_qs.evaluate(X_qs_test, y_test)

191/191 [==============================] - 1s 4ms/step - loss: 0.3572 - accuracy: 0.9954


[0.35717785358428955, 0.9953970313072205]

In [83]:
model.save('model_full.h5')
model_hmc.save('model_hmc.h5')
model_qs.save('model_qs.h5')

In [84]:
import pickle

In [85]:
pickle.dump(X_full_train, open('X_full_train.pkl', 'wb'))
pickle.dump(X_full_test, open('X_full_test.pkl', 'wb'))
pickle.dump(X_hmc_train, open('X_hmc_train.pkl', 'wb'))
pickle.dump(X_hmc_test, open('X_hmc_test.pkl', 'wb'))
pickle.dump(X_qs_train, open('X_qs_train.pkl', 'wb'))
pickle.dump(X_qs_test, open('X_qs_test.pkl', 'wb'))
pickle.dump(y_train, open('y_train.pkl', 'wb'))
pickle.dump(y_test, open('y_test.pkl', 'wb'))